In [1]:
from numba import cuda
import matplotlib.pyplot as plt
import numpy as np
import time
import math
from PIL import Image


In [2]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
im = plt.imread("/content/drive/MyDrive/Colab Notebooks/image.jpg")

In [5]:
shape = np.shape(im)
shape

(1080, 1920, 3)

In [6]:
devdata = cuda.to_device(im)
devOuput = cuda.device_array(shape, np.uint8)

In [16]:

@cuda.jit
def grayscale(src, dst):
  tidx = cuda.threadIdx.x + cuda.blockIdx.x * cuda.blockDim.x
  tidy = cuda.threadIdx.y + cuda.blockIdx.y * cuda.blockDim.y

  g = np.uint8((src[tidx, tidy, 0] + src[tidx, tidy, 1] + src[tidx, tidy, 2]) / 3)
  dst[tidx, tidy, 0] = dst[tidx, tidy, 1] = dst[tidx, tidy, 2] = g

In [17]:
t3 = time.time()

blockSize = (32, 32)
gridSize = (math.ceil(shape[0]/blockSize[0]),math.ceil(shape[1]/blockSize[1]))
grayscale[gridSize, blockSize](devdata, devOuput)

t4 = time.time()

print(t4-t3)


0.15127801895141602


In [19]:
imgray = devOuput.copy_to_host()
imgpu = Image.fromarray(imgray)
imgpu.save("/content/drive/MyDrive/Colab Notebooks/image_gray_GPU_2.jpeg")